In [3]:
!pip install pyarrow

     |████████████████████████████████| 42.3 MB 9.1 kB/s eta 0:00:01


In [3]:
import pandas as pd

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


from sklearn.metrics import mean_squared_error

In [5]:
#Question-1 : Downloading the Dataset
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')


total_records = df.shape[0]

In [6]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

#validation dataset
df_val.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df_val.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

#Finding the difference between dropoff and pickup
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime

#Finding the duration in minutes
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)


In [7]:
#Question-2: Computing the Standard deviation for the duration


df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [8]:
#Question-3: Finding the rides with duration between 1 and 60 minutes 
filtered_df = df[(df.duration >= 1) & (df.duration <= 60)]

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]


filtered_records = filtered_df.shape[0]

fraction_left = (filtered_records/total_records)*100

print(fraction_left)

98.1220282212598


In [9]:
#Question-4: One-hot coding choosing the categorical values

df = filtered_df

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']


df[categorical] = df[categorical].astype(str)



df_val[categorical] = df_val[categorical].astype(str)

#Turning the dataframe into list of dictnoraies
train_dicts = df[categorical + numerical].to_dict(orient='records')

#Fit a dictionary vectorizer
dv = DictVectorizer()

#Feature Matrix
X_train = dv.fit_transform(train_dicts)

#preparing the validation dict  values
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)



In [10]:
X_train.shape

(3009173, 516)

In [11]:
# Ques5:  Training a plan linear regression model

#Duration as response variable
target = 'duration'
y_train = df[target].values

#Model
lr = LinearRegression()
lr.fit(X_train, y_train)

#Prediction using model
y_pred = lr.predict(X_train)

rmse = mean_squared_error(y_train, y_pred, squared=False)

print(rmse)

7.649143388169879


In [12]:
#Ques6: Evaluating model using february data 

target = 'duration'
y_train = df[target].values
y_val = df_val[target].values


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse = mean_squared_error(y_val, y_pred, squared=False)

print(rmse)

11.873431677594954
